# 네이버 웹툰 댓글 수집하기

## 1. 하나의 웹툰 정해서 댓글 수집하기

In [1]:
import requests
import json
from tqdm.notebook import tqdm

### 1-1) 웹툰 하나의 최신 화의 베스트 댓글 수집하기

보통 요청 거절 당하면, 응답코드 : 404, 403, 402, 503 등
- 특수한 케이스 : 응답 코드 : 200인데, 반환값에 값이 안들어가있을 수 있음

In [2]:
url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_cv=20250508192749&_callback=jQuery363035249366563582263_1747223660380&lang=ko&country=KR&objectId=835030_40&categoryId=&pageSize=15&indexSize=10&groupId=835030&listType=OBJECT&pageType=more&page=1&initialize=true&followSize=5&userType=&useAltSort=true&replyPageSize=10&_=1747223660382"
resp = requests.get(url)
resp, resp.text # 응답 코드 : 200 but, 실제 반환값에는 status 403

(<Response [200]>,
 '<!doctype html><html lang="en"><head><title>HTTP Status 403 – Forbidden</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 403 – Forbidden</h1></body></html>')

In [3]:
url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_cv=20250508192749&_callback=jQuery363035249366563582263_1747223660380&lang=ko&country=KR&objectId=835030_40&categoryId=&pageSize=15&indexSize=10&groupId=835030&listType=OBJECT&pageType=more&page=1&initialize=true&followSize=5&userType=&useAltSort=true&replyPageSize=10&_=1747223660382"
headers = {
    'Referer':'https://comic.naver.com/webtoon/detail?titleId=783053&no=125&week=tu',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}

resp = requests.get(url, headers=headers)
resp.text

'jQuery363035249366563582263_1747223660380({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"835030_40","categoryId":"33_250422114534","templateId":"cut_v2","commentNo":"478222295","parentCommentNo":"478222295","replyLevel":1,"replyCount":6,"replyAllCount":6,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1745330721159,"contents":"아니 그것보다 너 뒤통수에 금이 갔어\\n병원부터 가ㅠㅠ","userIdNo":"Tdta","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"피아","userProfileImage":null,"profileType":"comic","modTime":"2025-04-22T23:05:21+0900","modTimeGmt":"2025-04-22T14:05:21+0000","regTime":"2025-04-22T23:05:21+0900","regTimeGmt":"2025-04-22T14:05:21+0000","sympathyCount":5305,"antipathyCount":12,"hideReplyButton":false,"st

In [4]:
len('jQuery363035249366563582263_1747223660380(')

42

In [5]:
jsn = json.loads(resp.text[42:-2]) # 순수 json 이어야 함. '{~~~}'
jsn

# 'result' > 'commentList' > []
#     'contents'

commentList = jsn['result']['commentList']
len(commentList)

15

In [6]:
for comment in commentList:
    print(comment['contents'])

아니 그것보다 너 뒤통수에 금이 갔어
병원부터 가ㅠㅠ
기억을 잃어도 본능은 여전하네
이 웹툰...
장르가 도대체 뭐죠?
아 그건 ㅇㅈ
아 죄없는 눈사람은 왜 부수는데
흐름뭐지(?)
슈퍼 이끌림⭐️
깨진거 아니냐고 ㅋㅋㅋㅋㅋㅋㅋ케
눈사람 부수는 사람 정떨어짐
난 이런 장르가 좋음
걍 작가가 그리면 다 되는 그런 세계관
뇌를 크게 안 써도 되고 즐겁기만함
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 이걸 또 상황극으로 받아들이네
이번 화는 뭐야 대체...
????
이래서 19금이었어....?
아 ㅋㅋㅋ 남매설정 참나ㅋㅋ


### 1-2) 웹툰 하나의 최신 화의 전체 댓글 수집하기
- 페이지가 여러개 > url이 여러개
- 끝 페이지 url을 알아내야한다.

In [7]:
import requests
import json
from tqdm.notebook import tqdm

In [8]:
url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_cv=20250508192749&_callback=jQuery36302833791072625951_1747224277998&lang=ko&country=KR&objectId=835030_40&categoryId=&pageSize=15&indexSize=10&groupId=835030&listType=OBJECT&pageType=more&page=1&currentPage=1&refresh=true&sort=new&_=1747224278002"
resp = requests.get(url)
resp, resp.text # 응답 코드 : 200 but, 실제 반환값에는 status 403

(<Response [200]>,
 '<!doctype html><html lang="en"><head><title>HTTP Status 403 – Forbidden</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 403 – Forbidden</h1></body></html>')

In [9]:
url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_cv=20250508192749&_callback=jQuery36302833791072625951_1747224277998&lang=ko&country=KR&objectId=835030_40&categoryId=&pageSize=15&indexSize=10&groupId=835030&listType=OBJECT&pageType=more&page=1&currentPage=1&refresh=true&sort=new&_=1747224278002"
headers = {
    'Referer':'https://comic.naver.com/webtoon/detail?titleId=783053&no=125&week=tu',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}

resp = requests.get(url, headers=headers)
resp.text

'jQuery36302833791072625951_1747224277998({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"835030_40","categoryId":"*","templateId":"default_v2","commentNo":"478987862","parentCommentNo":"478987862","replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1747547753117,"contents":"아닠ㅋㅋㅋㅋ꾸금 맞냐곸ㅋㅋㅋㅋ","userIdNo":"2cWjxy","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"이예인","userProfileImage":null,"profileType":"comic","modTime":"2025-05-18T14:55:53+0900","modTimeGmt":"2025-05-18T05:55:53+0000","regTime":"2025-05-18T14:55:53+0900","regTimeGmt":"2025-05-18T05:55:53+0000","sympathyCount":0,"antipathyCount":0,"hideReplyButton":false,"status":0,"mine":false,"best"

In [10]:
len('jQuery36302833791072625951_1747224277998(')

41

In [11]:
jsn = json.loads(resp.text[41:-2]) # 순수 json 이어야 함. '{~~~}'
jsn

# 'result' > 'commentList' > []
#     'contents'

commentList = jsn['result']['commentList']
len(commentList)

15

In [12]:
for comment in commentList:
    print(comment['contents'])

아닠ㅋㅋㅋㅋ꾸금 맞냐곸ㅋㅋㅋㅋ
개웃기다
눈 사람 건들지마라,,,돌 넣어놓기전에ㅡㅡ,,
엇...갑자기 사람이 달라보임..(negative)
완결뭐임.....??????,
ㅇㅈ한다
에바
사이다ㅋㅋㅋㅋ
너무 웃겨!!!
ㅋㅋㅋㅋㅋ 너무 귀여운데
이 정도면 ct 찍어야겠는데
괜히 부부가 아니네ㅋㅋㅋㅋ
네?
약간 나선환 같은디요
???? 눈 요정이라니


In [13]:
contents = []
totalPages = 45

for i in tqdm(range(totalPages)):
    page = i+1
    url = f"https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_cv=20250508192749&_callback=jQuery36302833791072625951_1747224277998&lang=ko&country=KR&objectId=835030_40&categoryId=&pageSize=15&indexSize=10&groupId=835030&listType=OBJECT&pageType=more&page=1&currentPage={page}&refresh=true&sort=new&_=1747224278002"
    headers = {
              'Referer':'https://comic.naver.com/webtoon/detail?titleId=783053&no=125&week=tu',
              'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
              }

    resp = requests.get(url, headers=headers)

    jsn = resp.text[resp.text.find("(")+1 : resp.text.rfind(")")]
    dic_resp = json.loads(jsn)

    commentList = dic_resp['result']['commentList']

    for comment in commentList:
        contents.append(comment['contents'])
    # 'result'
    # 'commentList'
    # []
    # 'contents'

len(contents)

  0%|          | 0/45 [00:00<?, ?it/s]

675

In [14]:
for c in contents:
    print(c)

아닠ㅋㅋㅋㅋ꾸금 맞냐곸ㅋㅋㅋㅋ
개웃기다
눈 사람 건들지마라,,,돌 넣어놓기전에ㅡㅡ,,
엇...갑자기 사람이 달라보임..(negative)
완결뭐임.....??????,
ㅇㅈ한다
에바
사이다ㅋㅋㅋㅋ
너무 웃겨!!!
ㅋㅋㅋㅋㅋ 너무 귀여운데
이 정도면 ct 찍어야겠는데
괜히 부부가 아니네ㅋㅋㅋㅋ
네?
약간 나선환 같은디요
???? 눈 요정이라니
아닠ㅋㅋㅋㅋ꾸금 맞냐곸ㅋㅋㅋㅋ
개웃기다
눈 사람 건들지마라,,,돌 넣어놓기전에ㅡㅡ,,
엇...갑자기 사람이 달라보임..(negative)
완결뭐임.....??????,
ㅇㅈ한다
에바
사이다ㅋㅋㅋㅋ
너무 웃겨!!!
ㅋㅋㅋㅋㅋ 너무 귀여운데
이 정도면 ct 찍어야겠는데
괜히 부부가 아니네ㅋㅋㅋㅋ
네?
약간 나선환 같은디요
???? 눈 요정이라니
아닠ㅋㅋㅋㅋ꾸금 맞냐곸ㅋㅋㅋㅋ
개웃기다
눈 사람 건들지마라,,,돌 넣어놓기전에ㅡㅡ,,
엇...갑자기 사람이 달라보임..(negative)
완결뭐임.....??????,
ㅇㅈ한다
에바
사이다ㅋㅋㅋㅋ
너무 웃겨!!!
ㅋㅋㅋㅋㅋ 너무 귀여운데
이 정도면 ct 찍어야겠는데
괜히 부부가 아니네ㅋㅋㅋㅋ
네?
약간 나선환 같은디요
???? 눈 요정이라니
아닠ㅋㅋㅋㅋ꾸금 맞냐곸ㅋㅋㅋㅋ
개웃기다
눈 사람 건들지마라,,,돌 넣어놓기전에ㅡㅡ,,
엇...갑자기 사람이 달라보임..(negative)
완결뭐임.....??????,
ㅇㅈ한다
에바
사이다ㅋㅋㅋㅋ
너무 웃겨!!!
ㅋㅋㅋㅋㅋ 너무 귀여운데
이 정도면 ct 찍어야겠는데
괜히 부부가 아니네ㅋㅋㅋㅋ
네?
약간 나선환 같은디요
???? 눈 요정이라니
아닠ㅋㅋㅋㅋ꾸금 맞냐곸ㅋㅋㅋㅋ
개웃기다
눈 사람 건들지마라,,,돌 넣어놓기전에ㅡㅡ,,
엇...갑자기 사람이 달라보임..(negative)
완결뭐임.....??????,
ㅇㅈ한다
에바
사이다ㅋㅋㅋㅋ
너무 웃겨!!!
ㅋㅋㅋㅋㅋ 너무 귀여운데
이 정도면 ct 찍어야겠는데
괜히 부부가 아니네ㅋㅋㅋㅋ
네?
약간 나선환 같은디요
???? 눈 요정이라니
아닠ㅋㅋㅋㅋ꾸금 맞냐곸ㅋㅋㅋㅋ
개웃기다
눈 사람 건들지마라,,,돌 넣어놓

### 1-3) 웹툰 하나의 모든 화의 베스트 댓글 수집하기

  0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
contents

['이거 원래 태준이형이 미X아빠로 제목 지었는데 \n\n김부장으로 제목 바뀌었네 외지주 김부장과거 이야기 맞음',
 '박태준 만화 특징 : 양산형이라고 욕하는데 존.나 재밌음',
 '아빠 몸이 저런데 딸은 평범한 회사원인줄 안건가?',
 '이거 개 심각한 문제인게 정종택 작가는 작품이 아예 기록 말소 처리 당하고 연재 자격을 박탈 당했던 사람임 그 정도로 저지른 짓이 엄청 났다는 말이고 근데 그런 사람을 떡 하니 다시 불러 정식 작가 자리에 앉혔다는 건 그 때의 그건 전부 당장 커버 처 주긴 힘드니 겉으론 네이버 웹툰은 저작권 법을 존중 합니다 쇼를 하고 뒤 에선 아 언제 다시 정식 작가로 써 먹지 하고 각을 재고 있었다는 해석도 충분히 가능하다는 거임 웹툰 독자 및 정식 작가를 꿈 꾸는 작가 지망생들에게 전부 빅엿을 먹인거나 다름 없음',
 '이제 외지주 김부장 싸움씬 나올때마다 갈드컵 열리겠네',
 '양영순은 진짜 그렇게 욕먹어도 어쨌든 명목상 완결이라도 하고 갔지, 지각때문인거 같지만 미리보기로 쿠키 뜯지도 않았고 ㅋㅋ 근데 미리보기 열어두고 트래이싱으로 쫓겨난 작가를 다시 데리고오는것도 모자라서 연재까지 시키고 미리보기 또 열어뒀네? 만화 등장인물로 네이버웹툰 사장 써먹는거부터 친목질인건 느꼈는데 그냥 프로의식같은건 없는 환쟁이 판이었네 ㅋㅋㅋㅋ ',
 '트레이싱 언급 베댓 다 내리네 ㅋㅋㅋㅋㅋㅋㅋ 진짜 가지가지 한다 ',
 '딸년 말처하는거봐',
 '김성모랑 친한 박태준\n김성모는 복붙없인 못살고 슬램덩크를 트레이싱하여 나락간 작가임\n박태준은 월요일 신작 표지조차 외지주 표지의 복붙이고 신작 1화만에 복붙 4개이상쓰는 복붙의 화신이고\n이런 박태준이 트레이싱했던 작가랑 작업하는건 전혀 이상한 일이 아님 애초부터 박태준은 만화가가 아니라 공장장이기 때문에 복붙과 트레이싱은 안하면 손해라고 생각할정도의 마인드의 보유자이기 때문임\n가장 최근 미리보기 외지주에서 gd가 공격을 피하는 5가지의 얼굴들은 전부다 복붙일정도로 복붙으로 모든걸 해결하는 작가거든

### 1-4) 웹툰 하나의 모든 화의 전체 댓글 수집하기
- current_no loop + total pages loop : 2중 for문
- 하나의 화에서
    1. 해당 화의 total pages 가져오기  
    2. for total pages

## 2. 원하는 웹툰 입력 받아서 댓글 수집하기


In [ ]:
import requests
import json
from tqdm.notebook import tqdm

### 2-0) 웹툰 정보 DB 저장하기
- 원하는 웹툰을 입력했을 때, 해당 웹툰의 url을 가져와야 한다.
- 웹툰 정보 DB를 먼저 모아야 한다.
    - 제목
    - 링크
    - title_id
    - 최신화 no

In [ ]:
webtoon_db = {}

In [ ]:
wkdays = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']

In [ ]:
url = 'https://comic.naver.com/api/webtoon/titlelist/weekday?order=user'

{'titleListMap': {'SATURDAY': [{'titleId': 818791,
    'titleName': '개꿈',
    'author': '신송림',
    'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/818791/thumbnail/thumbnail_IMAG21_a613db8b-8319-4bfe-8971-1df89f4df979.jpg',
    'up': False,
    'rest': False,
    'bm': False,
    'adult': True,
    'starScore': 9.92804,
    'viewCount': 0,
    'openToday': False,
    'potenUp': False,
    'bestChallengeLevelUp': False,
    'finish': False,
    'new': False},
   {'titleId': 833255,
    'titleName': '낢이 사는 이야기 - 계속되는 미미한 인생',
    'author': '서나래',
    'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/833255/thumbnail/thumbnail_IMAG21_504f6152-8a60-42a0-a1c0-43cd77f9c430.jpg',
    'up': False,
    'rest': False,
    'bm': False,
    'adult': False,
    'starScore': 9.94683,
    'viewCount': 0,
    'openToday': False,
    'potenUp': False,
    'bestChallengeLevelUp': False,
    'finish': False,
    'new': True},
   {'titleId': 826670,
    'titleName': '아수라',
    'author': '류

In [ ]:
webtoon_db

{'뷰티풀 군바리': {'제목': '뷰티풀 군바리',
  'title_id': 648419,
  'link': 'https://comic.naver.com/webtoon/list?titleId=648419&tab=mon',
  'c_ no': 427},
 '참교육': {'제목': '참교육',
  'title_id': 758037,
  'link': 'https://comic.naver.com/webtoon/list?titleId=758037&tab=mon',
  'c_ no': 133},
 '백수세끼': {'제목': '백수세끼',
  'title_id': 733074,
  'link': 'https://comic.naver.com/webtoon/list?titleId=733074&tab=mon',
  'c_ no': 200},
 '신의 탑': {'제목': '신의 탑',
  'title_id': 183559,
  'link': 'https://comic.naver.com/webtoon/list?titleId=183559&tab=mon',
  'c_ no': 608},
 '퀘스트지상주의': {'제목': '퀘스트지상주의',
  'title_id': 783052,
  'link': 'https://comic.naver.com/webtoon/list?titleId=783052&tab=mon',
  'c_ no': 126},
 '장씨세가 호위무사': {'제목': '장씨세가 호위무사',
  'title_id': 728750,
  'link': 'https://comic.naver.com/webtoon/list?titleId=728750&tab=mon',
  'c_ no': 253},
 '윈드브레이커': {'제목': '윈드브레이커',
  'title_id': 602910,
  'link': 'https://comic.naver.com/webtoon/list?titleId=602910&tab=mon',
  'c_ no': 484},
 '소녀의 세계': {'제목': '소녀의 세

### 2-1) 원하는 웹툰 입력 받아 최신화의 베스트 댓글 수집하기

In [ ]:
toon_title = input('어떤 웹툰의 댓글을 수집할까요?')

webtoon_db[toon_title]


### 2-2) 아래 사항 선택 가능하도록 함수화하기
- 원하는 웹툰
- 최신 에피소드 / 전체 에피소드 여부
- 베스트 댓글 / 전체 댓글 여부

In [ ]:
def wbtn_crawling(title, current_only=True, best_only=True): # False : total


In [ ]:
# 원하는 입력 파라미터 설정할 때 정상작동하는지 확인
wbtn_crawling()

## 3. 전체 웹툰의 최신화의 베스트 댓글 가져오기

In [ ]:
import requests
import json
from tqdm.notebook import tqdm

#### 3-0) 모든 웹툰 정보 DB 저장하기
- 웹툰 정보 DB를 먼저 모아야 한다.
    - 제목
    - 링크
    - title_id
    - 최신화 no

### 3-1) 모든 웹툰의 최신화의 베스트 댓글 수집하기

### 3-2) 아래 사항을 선택 가능하도록 함수화하기